<img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400>  </a>

<h1 align=center><font size = 5>Peer-graded Assignment: Build a Regression Model in Keras</font></h1>

# Introduction

In this course project, you will build a regression model using the deep learning Keras library, and then you will experiment with increasing the number of training epochs and changing number of hidden layers and you will see how changing these parameters impacts the performance of the model.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item1">Build a baseline model</a>  
2. <a href="#item2">Normalize the data</a>  
3. <a href="#item3">Increase the number of epochs</a>  
4. <a href="#item4">Increase the number of hidden layers</a>  

</font>
</div>

<a id="item1"></a>
# Build a baseline model

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error as the loss function.

Steps:
1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the `train_test_split` helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the `mean_squared_error` function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

### Load the dataset

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them.

In [1]:
import pandas as pd
import numpy as np

## load the data from source
df = pd.read_csv('https://cocl.us/concrete_data')

print(df.shape)
df.head()

(1030, 9)


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check the data structure and quality

In [2]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [3]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data have different scales and there are not any missing values

**Split data into predictors and target**

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [4]:
## split data into predictors and target
X = df.drop("Strength", axis=1)
y = df["Strength"].copy()

print(X.shape)
print(y.shape)

(1030, 8)
(1030,)


### Build DNN model

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

def build_dnn(n_layers, n_nodes, n_inputs, activation_func="relu"):
    """
    build deep neural network
    """
    model = Sequential()
    
    ## input layer
    model.add(InputLayer(input_shape=(n_inputs,)))
    
    ## hidden layers
    for layer in range(n_layers):
        model.add(Dense(n_nodes, activation=activation_func))
        
    ## output layer
    model.add(Dense(1))
    
    ## compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    ## print model summary
    model.summary()
    
    return model

Using TensorFlow backend.


### Train and Evaluate DNN model

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def model_train(X, y, model, n_iterations=50, n_epochs=50):
    """
    train and evaluate the model on different train-test splits
    """
    mse = []

    for i in range(n_iterations):
        
        ## split the data set into train and test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        ## train the model
        model.fit(X_train, y_train, epochs=n_epochs, verbose=0)
        
        ## calculate score
        y_pred = model.predict(X_test)
        mse.append(mean_squared_error(y_test, y_pred))
    
    print("the mean is {} and the standard deviation is {}".format(round(np.mean(mse),4), round(np.std(mse),4)))
    
    return mse

In [7]:
## create a dictionary to store the scores for each project part
scores={}

In [8]:
keras.backend.clear_session()

## build neural network
shallow_model = build_dnn(n_layers=1, n_nodes=10, n_inputs=X.shape[1])

## train model
scores["A"] = model_train(X, y, shallow_model)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
the mean is 67.7292 and the standard deviation is 27.2505


<a id="item2"></a>

# Normalize the Data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [9]:
X_norm = (X - X.mean()) / X.std()
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
keras.backend.clear_session()

## build neural network
shallow_model = build_dnn(n_layers=1, n_nodes=10, n_inputs=X.shape[1])

## train model
scores["B"] =  model_train(X_norm, y, shallow_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
the mean is 48.7866 and the standard deviation is 53.9284


The mean of the mean squared errors has decreased compared to the value from part A. Data normalization has an impact.

<a id="item3"></a>

# Increase the number of epochs

Repeat Part B but use 100 epochs this time for training.

In [11]:
keras.backend.clear_session()

## build neural network
shallow_model = build_dnn(n_layers=1, n_nodes=10, n_inputs=X.shape[1])

## train model
scores["C"] =  model_train(X_norm, y, shallow_model, n_epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
the mean is 42.2112 and the standard deviation is 23.4988


The mean of the mean squared errors has decreased compared to the value from part B. Longer training results to better results.

<a id="item4"></a>

# Increase the number of hidden layers

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

In [12]:
keras.backend.clear_session()

## build deep neural network
deep_model = build_dnn(n_layers=3, n_nodes=10, n_inputs=X.shape[1], activation_func="relu")

## train
scores["D"] =  model_train(X_norm, y, deep_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________
the mean is 28.7167 and the standard deviation is 15.9229


The mean of the mean squared errors has decreased compared to the value from part B. Deeper neural network results to better results.